In [25]:
!pip install Flask pyngrok line-bot-sdk requests --quiet
!pip install google-genai --quiet

In [26]:
from google.colab import userdata

ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
line_channel_access_token = userdata.get('LINE_CHANNEL_ACCESS_TOKEN')
line_channel_secret = userdata.get('LINE_CHANNEL_SECRET')
gemini_api_key = userdata.get('GEMINI_API_KEY')
port = 5051

In [27]:
import os
from pyngrok import ngrok


In [28]:
ngrok.kill()

In [29]:
import requests

ngrok.set_auth_token(ngrok_authtoken)
tunnel = ngrok.connect(5051, name="linebot_tunnel")
webhook_url = tunnel.public_url

print(f"Ngrok URL: {webhook_url}")

# 自動更新 LINE Webhook URL
def update_line_webhook(webhook_url):
    """使用 LINE Messaging API 更新 Webhook URL"""
    url = "https://api.line.me/v2/bot/channel/webhook/endpoint"
    headers = {
        "Authorization": f"Bearer {line_channel_access_token}",
        "Content-Type": "application/json"
    }
    data = {
        "endpoint": webhook_url
    }

    response = requests.put(url, headers=headers, json=data)

    if response.status_code == 200:
        print(f"✅ LINE Webhook URL 已自動更新為：{webhook_url}")
        return True
    else:
        print(f"❌ 更新失敗：{response.status_code} - {response.text}")
        return False

# 執行更新
update_line_webhook(webhook_url)

Ngrok URL: https://noncalculative-semireactionary-debra.ngrok-free.dev
✅ LINE Webhook URL 已自動更新為：https://noncalculative-semireactionary-debra.ngrok-free.dev


True

In [30]:
from google import genai
from google.genai.types import Tool, GenerateContentConfig, GoogleSearch

# === 初始化 Google Gemini ===
client = genai.Client(api_key=gemini_api_key)

google_search_tool = Tool(
   google_search=GoogleSearch()
)

chat = client.chats.create(
    model="gemini-2.5-flash",
    config=GenerateContentConfig(
        system_instruction="你是一個中文的AI助手，請用繁體中文回答",
        tools=[google_search_tool],
        response_modalities=["TEXT"],
    )
)

In [31]:
def stateful_query(payload):
    response = chat.send_message(message=payload)
    return response.text

In [32]:
result = stateful_query("簡介JET SL SuperC (50字以內)")
print(result)

SYM JET SL SuperC 是三陽機車結合賽道基因的運動速克達。它搭載新一代水冷高壓縮比引擎及競能電驅系統，大幅提升加速性能與燃油效率。配備ABS、TCS，提供卓越操控與行車安全。


In [33]:
result2 = stateful_query("JET SL SuperC是什麼？(50字以內)")
print(result2)

SYM JET SL SuperC 是三陽機車旗下的運動速克達。它搭載水冷引擎與多項電控系統，旨在提供優異的操控性與加速性能，並強調安全性。


In [ ]:
from flask import Flask, request, abort
import logging
import os
import time
from google.genai import types

from linebot.v3 import (
    WebhookHandler
)
from linebot.v3.exceptions import (
    InvalidSignatureError
)
from linebot.v3.messaging import (
    Configuration,
    ApiClient,
    MessagingApi,
    MessagingApiBlob,
    ReplyMessageRequest,
    TextMessage
)
from linebot.v3.webhooks import (
    MessageEvent,
    TextMessageContent,
    FileMessageContent
)

app = Flask(__name__)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)
app.logger.setLevel(logging.INFO)

configuration = Configuration(access_token=line_channel_access_token)
handler = WebhookHandler(line_channel_secret)

# 儲存檔案的目錄
UPLOAD_DIR = "/content/uploaded_files"
os.makedirs(UPLOAD_DIR, exist_ok=True)

# 儲存每個使用者的對話 session 和上傳的檔案
user_sessions = {}  # {user_id: {"chat": chat_object, "uploaded_file": gemini_file}}

def get_user_session(user_id):
    """取得或建立使用者的對話 session"""
    if user_id not in user_sessions:
        # 建立新的對話 session
        new_chat = client.chats.create(
            model="gemini-2.5-flash",
            config=GenerateContentConfig(
                system_instruction="你是一個中文的AI助手，請用繁體中文回答。如果使用者有提供參考文件，請根據文件內容回答問題。",
                tools=[google_search_tool],
                response_modalities=["TEXT"],
            )
        )
        user_sessions[user_id] = {
            "chat": new_chat,
            "uploaded_file": None
        }
    return user_sessions[user_id]

def download_line_file(message_id, file_name):
    """從 LINE 下載使用者上傳的檔案"""
    with ApiClient(configuration) as api_client:
        line_bot_blob_api = MessagingApiBlob(api_client)
        file_content = line_bot_blob_api.get_message_content(message_id)

        file_path = os.path.join(UPLOAD_DIR, file_name)

        with open(file_path, 'wb') as f:
            f.write(file_content)

        return file_path

def upload_file_to_gemini(file_path):
    """上傳檔案到 Gemini Files API"""
    uploaded_file = client.files.upload(
        file=file_path,
        config={'display_name': os.path.basename(file_path)}
    )

    # 等待檔案處理完成
    while uploaded_file.state.name == "PROCESSING":
        print("檔案處理中...")
        time.sleep(1)
        uploaded_file = client.files.get(name=uploaded_file.name)

    if uploaded_file.state.name == "FAILED":
        raise Exception("檔案上傳處理失敗")

    return uploaded_file

def query_with_rag(user_id, question):
    """使用 RAG 模式回答問題"""
    session = get_user_session(user_id)
    uploaded_file = session["uploaded_file"]

    if uploaded_file:
        # 有上傳檔案，使用 RAG 模式
        response = client.models.generate_content(
            model="gemini-2.5-flash",
            contents=[
                types.Content(
                    role="user",
                    parts=[
                        types.Part.from_uri(
                            file_uri=uploaded_file.uri,
                            mime_type=uploaded_file.mime_type
                        ),
                        types.Part.from_text(text=f"請根據上述提供的檔案內容，用繁體中文回答這個問題：{question}")
                    ]
                )
            ]
        )
        return response.text
    else:
        # 沒有上傳檔案，使用一般多輪對話
        response = session["chat"].send_message(message=question)
        return response.text

@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    print("BODY: ", body)
    app.logger.info("Request body: " + body)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        app.logger.info("Invalid signature.")
        abort(400)

    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    """處理文字訊息"""
    text = event.message.text
    user_id = event.source.user_id

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        if text.startswith('AI '):
            prompt = text[3:]
            try:
                # 使用 RAG 或一般對話
                reply_text = query_with_rag(user_id, prompt)

                # 檢查是否有上傳檔案，加上提示
                session = get_user_session(user_id)
                if session["uploaded_file"]:
                    reply_text = f"📄 [RAG 模式]\n\n{reply_text}"

                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=reply_text)]
                    )
                )
            except Exception as e:
                line_bot_api.reply_message_with_http_info(
                    ReplyMessageRequest(
                        reply_token=event.reply_token,
                        messages=[TextMessage(text=f"❌ 發生錯誤：{str(e)}")]
                    )
                )
        elif text == "清除文件":
            # 清除使用者上傳的檔案
            session = get_user_session(user_id)
            session["uploaded_file"] = None
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="✅ 已清除上傳的文件，恢復一般對話模式。")]
                )
            )
        else:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="請輸入「AI 問題」來開始對話\n或上傳 TXT/PDF 檔案啟用 RAG 模式")]
                )
            )

@handler.add(MessageEvent, message=FileMessageContent)
def handle_file_message(event):
    """處理使用者上傳的檔案"""
    user_id = event.source.user_id
    file_name = event.message.file_name

    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        # 檢查檔案類型
        if not (file_name.endswith('.txt') or file_name.endswith('.pdf')):
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text="⚠️ 目前只支援 TXT 或 PDF 檔案")]
                )
            )
            return

        try:
            # 下載檔案
            file_path = download_line_file(event.message.id, file_name)
            print(f"檔案已下載：{file_path}")

            # 上傳到 Gemini
            uploaded_file = upload_file_to_gemini(file_path)
            print(f"檔案已上傳到 Gemini：{uploaded_file.uri}")

            # 儲存到使用者 session
            session = get_user_session(user_id)
            session["uploaded_file"] = uploaded_file

            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"✅ 檔案「{file_name}」上傳成功！\n\n現在您可以輸入「AI 問題」來詢問關於這份文件的問題。\n\n輸入「清除文件」可恢復一般對話模式。")]
                )
            )
        except Exception as e:
            line_bot_api.reply_message_with_http_info(
                ReplyMessageRequest(
                    reply_token=event.reply_token,
                    messages=[TextMessage(text=f"❌ 檔案處理失敗：{str(e)}")]
                )
            )

if __name__ == "__main__":
    app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5051
INFO:werkzeug:Press CTRL+C to quit
INFO:__main__:Request body: {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"file","id":"594455660268093495","markAsReadToken":"qxDmfGCx3ReFvvENTVCH_ax368H8y0UVOyPk8JJMR2mLlCS8JTdpDREU0SGXV0D7jirU5kw1aaihPxevI2XiCrnoWUuAX1yAOSHfP9_Y6AGSy-KxNrukrMwd-FTxUt-ft30Co7_4FJy91AkUbQQEkaoiVnjsaZE-6SYzGYdLIOe_3xmJBdsJkvCWTE-ZwWY9sC9lpkiCzS257SbCF3f1PQ","fileName":"faq.txt","fileSize":96,"contentProvider":{"type":"line"}},"webhookEventId":"01KDS9V60X47Z22NZ5WV66R4G2","deliveryContext":{"isRedelivery":false},"timestamp":1767154488867,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"2f61e6ebf19c43d2b4a3da1301cdb3bb","mode":"active"}]}


BODY:  {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"file","id":"594455660268093495","markAsReadToken":"qxDmfGCx3ReFvvENTVCH_ax368H8y0UVOyPk8JJMR2mLlCS8JTdpDREU0SGXV0D7jirU5kw1aaihPxevI2XiCrnoWUuAX1yAOSHfP9_Y6AGSy-KxNrukrMwd-FTxUt-ft30Co7_4FJy91AkUbQQEkaoiVnjsaZE-6SYzGYdLIOe_3xmJBdsJkvCWTE-ZwWY9sC9lpkiCzS257SbCF3f1PQ","fileName":"faq.txt","fileSize":96,"contentProvider":{"type":"line"}},"webhookEventId":"01KDS9V60X47Z22NZ5WV66R4G2","deliveryContext":{"isRedelivery":false},"timestamp":1767154488867,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"2f61e6ebf19c43d2b4a3da1301cdb3bb","mode":"active"}]}
檔案已下載：/content/uploaded_files/faq.txt
檔案已上傳到 Gemini：https://generativelanguage.googleapis.com/v1beta/files/9s00wj7dnyxw


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 04:14:52] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594455793747623937","quoteToken":"2Gz8ADlUkSKCWpIqKf7q_yxO94XhtfmkmfnS6jqZ2R02Zpd2eXl2EhPy0PWKQuuVXIHg6zPsejTHxt7XP-twsvP6aF5AjhBQBSI-nnUt4stUxzJbTa6V6uilN-Y1-3EF__HnqmDtGthUv1_0cD6pKA","markAsReadToken":"PYJmvvBWv05bK3H_Zb31I3jtocp6L60pNhMbLKhwNOBjA-ul1jVCDQmK0c-pPbYItGNH_i8Pi0BXnEI6ejEtIaMYw3umiVTEo6Y3DOcv2wnC1lyR5ZTUEVnWDBJAmNsDo19C8WAVq642NeBwt-1umeqeOh486m9ERhI0N9Gyboi8hU2L_RaBK0c0j87vTHLLDMdTfelXbCYmmOTer8rdaQ","text":"AI 簡介JET SL Super C (50字以內) "},"webhookEventId":"01KDS9XKGQZC2HRASA5TJVSJ9A","deliveryContext":{"isRedelivery":false},"timestamp":1767154568218,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"aae1bc1721cd4f5b9515403113a50fe0","mode":"active"}]}


BODY:  {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594455793747623937","quoteToken":"2Gz8ADlUkSKCWpIqKf7q_yxO94XhtfmkmfnS6jqZ2R02Zpd2eXl2EhPy0PWKQuuVXIHg6zPsejTHxt7XP-twsvP6aF5AjhBQBSI-nnUt4stUxzJbTa6V6uilN-Y1-3EF__HnqmDtGthUv1_0cD6pKA","markAsReadToken":"PYJmvvBWv05bK3H_Zb31I3jtocp6L60pNhMbLKhwNOBjA-ul1jVCDQmK0c-pPbYItGNH_i8Pi0BXnEI6ejEtIaMYw3umiVTEo6Y3DOcv2wnC1lyR5ZTUEVnWDBJAmNsDo19C8WAVq642NeBwt-1umeqeOh486m9ERhI0N9Gyboi8hU2L_RaBK0c0j87vTHLLDMdTfelXbCYmmOTer8rdaQ","text":"AI 簡介JET SL Super C (50字以內) "},"webhookEventId":"01KDS9XKGQZC2HRASA5TJVSJ9A","deliveryContext":{"isRedelivery":false},"timestamp":1767154568218,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"aae1bc1721cd4f5b9515403113a50fe0","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 04:16:14] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594455871795233282","quoteToken":"A8GhaG9R5hFcYetnUymXN5qzjZtWtpIqWpw5Bj718e_TYpP-AfclHpzN9BZXL9YxYl3OezlCg2BlntOQASChLGj7RmyVtPlaJeZP5ienZg8U_gvmiA7zOPSuoEHNfUW6btUrqIlFk4_c8c4CQocM9g","markAsReadToken":"k5yE69vEtwBXZIV52IMWOX1a6qOkQHJdPDmQ75okjWTLOHtVziPOy3nU4zfsdDb4z1RtZjckzk6ZLEnpgZ6HWpI_l17vv9UoYrpHCte2ABQwSyk2tZa4BrlwKD6F91rznz9n-g5g0Sn3SU86jELHMvM6nFuFa5HBjPKQc3Q0w1Lbl7PTmN21_VzGeLLtSQCTnmzmdMOuoi49O1xAHMSDLQ","text":"JET SL SuperC是什麼?(50字以內)"},"webhookEventId":"01KDS9Z10ZNWNF5QV9GTNJ4TM2","deliveryContext":{"isRedelivery":false},"timestamp":1767154614850,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"ec2fdc6780b34fb18f71cd69ccba0c28","mode":"active"}]}


BODY:  {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594455871795233282","quoteToken":"A8GhaG9R5hFcYetnUymXN5qzjZtWtpIqWpw5Bj718e_TYpP-AfclHpzN9BZXL9YxYl3OezlCg2BlntOQASChLGj7RmyVtPlaJeZP5ienZg8U_gvmiA7zOPSuoEHNfUW6btUrqIlFk4_c8c4CQocM9g","markAsReadToken":"k5yE69vEtwBXZIV52IMWOX1a6qOkQHJdPDmQ75okjWTLOHtVziPOy3nU4zfsdDb4z1RtZjckzk6ZLEnpgZ6HWpI_l17vv9UoYrpHCte2ABQwSyk2tZa4BrlwKD6F91rznz9n-g5g0Sn3SU86jELHMvM6nFuFa5HBjPKQc3Q0w1Lbl7PTmN21_VzGeLLtSQCTnmzmdMOuoi49O1xAHMSDLQ","text":"JET SL SuperC是什麼?(50字以內)"},"webhookEventId":"01KDS9Z10ZNWNF5QV9GTNJ4TM2","deliveryContext":{"isRedelivery":false},"timestamp":1767154614850,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"ec2fdc6780b34fb18f71cd69ccba0c28","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 04:16:56] "POST / HTTP/1.1" 200 -
INFO:__main__:Request body: {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594455936555548763","quoteToken":"qkU9tnm_2kojPyfyyfAl0JRE3AU20IJquk4IiIa_4PzgqGW1PFn7YlcOQ9y5tediSgAY-43H9gDqU60SUca8qgpHHmexY9BJ5uiqBtQPQJ83kH1xDdalxW3ke30FoUnhmIby0qSe9qDTDgVgwibgjw","markAsReadToken":"spP8FPVCwBvF2TbXzTBZ-syroXSGOraQkh9Wb36DerMgXGvy2sIcrG8sI0AAmf5s07m52ZuOSCE2NgHSdBZu0aLYxnqXn5o8FqPje1MG641RBHzahBfjnmn-CdamFCDTE-_PM8cdpcWaw264tIU2H3V18v1VcTNvdzAxBzxZgsvf8PTpa_N-88hzseD3YYF0yWTBY6D8LwF6WgZvQxqLdg","text":"AI JET SL SuperC是什麼?(50字以內)"},"webhookEventId":"01KDSA06P6A3N2JXSK6Z568WGC","deliveryContext":{"isRedelivery":false},"timestamp":1767154653386,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"058e38ad641442a08031a36319933851","mode":"active"}]}


BODY:  {"destination":"U1adbf8b40af0e85791b2a893f8ea9ac6","events":[{"type":"message","message":{"type":"text","id":"594455936555548763","quoteToken":"qkU9tnm_2kojPyfyyfAl0JRE3AU20IJquk4IiIa_4PzgqGW1PFn7YlcOQ9y5tediSgAY-43H9gDqU60SUca8qgpHHmexY9BJ5uiqBtQPQJ83kH1xDdalxW3ke30FoUnhmIby0qSe9qDTDgVgwibgjw","markAsReadToken":"spP8FPVCwBvF2TbXzTBZ-syroXSGOraQkh9Wb36DerMgXGvy2sIcrG8sI0AAmf5s07m52ZuOSCE2NgHSdBZu0aLYxnqXn5o8FqPje1MG641RBHzahBfjnmn-CdamFCDTE-_PM8cdpcWaw264tIU2H3V18v1VcTNvdzAxBzxZgsvf8PTpa_N-88hzseD3YYF0yWTBY6D8LwF6WgZvQxqLdg","text":"AI JET SL SuperC是什麼?(50字以內)"},"webhookEventId":"01KDSA06P6A3N2JXSK6Z568WGC","deliveryContext":{"isRedelivery":false},"timestamp":1767154653386,"source":{"type":"user","userId":"U4a7bcc8e7271a339e868a8b180a0945f"},"replyToken":"058e38ad641442a08031a36319933851","mode":"active"}]}


INFO:werkzeug:127.0.0.1 - - [31/Dec/2025 04:17:38] "POST / HTTP/1.1" 200 -
